In [4]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px

In [5]:
sns.set(
    font_scale =2,
    style      ="whitegrid",
    rc         ={'figure.figsize':(20,7)}
)

In [6]:
sales_team  = pd.read_csv('sales_team.csv')
order_leads = pd.read_csv('order_leads.csv')
invoices    = pd.read_csv('invoices.csv')

In [5]:
sales_team.head()

,Sales Rep,Sales Rep Id,Company Name,Company Id
0,William Taylor,ZTZA0ZLYZR85PTUJ,Melancholy Social-Role,D0AUXPP07H6AVSGD
1,William Taylor,ZTZA0ZLYZR85PTUJ,Pee-Wee Pigeon,TJW99R4RWH02INII
2,William Taylor,ZTZA0ZLYZR85PTUJ,Vaginal 36-Year-Old,3S20IO1O1DYXMUNU
3,William Taylor,ZTZA0ZLYZR85PTUJ,Unhurried Musician'S,FDWW0TQRPGMVI2A7
4,William Taylor,ZTZA0ZLYZR85PTUJ,Comprehensive Lists,C30FY83UNLVBA0O3


In [ ]:
sales_team = sales_team.rename(columns={'Company Name': 'Company_Name', 
                                        'Company Id'  : 'Company_Id', 
                                        'Sales Rep'   : 'Sales_Rep', 
                                        'Sales_Rep_Id': 'Sales Rep Id'})

In [ ]:
order_leads['Date'] = pd.to_datetime(order_leads.Date)

In [ ]:
order_leads = order_leads.rename(columns={'Order Id': 'Order_Id', 
                                          'Company Id': 'Company_Id', 
                                          'Company Name': 'Company_Name', 
                                          'Order Value': 'Order_Value'})

In [ ]:
order_leads_with_team_data = order_leads.merge(sales_team, how='left', on=['Company_Name', 'Company_Id'])

In [ ]:
order_leads_with_team_data.head()

In [ ]:
top_sales = order_leads_with_team_data.groupby(['Sales_Rep'], as_index=False) \
    .agg({'Converted': 'mean'}) \
    .sort_values('Converted') \
    .tail(10)

In [ ]:
all_sales = order_leads_with_team_data.groupby(['Sales_Rep'], as_index=False) \
    .agg({'Converted': 'mean'}) \
    .sort_values('Converted') 

In [ ]:
sns.distplot(all_sales.Converted, kde=False)

In [ ]:
ax = sns.barplot(data=top_sales, y='Converted', x='Sales_Rep')
ax.set_xticklabels(top_sales.Sales_Rep, rotation=45)
sns.despine()

In [ ]:
order_leads.head()

In [ ]:
# График CR по дням

In [ ]:
cr_by_day = order_leads.groupby(['Date']) \
    .agg({'Converted': 'mean'}) \
    .rename(columns={'Converted': 'CR'}) \
    .sort_values('Date')

In [ ]:
cr_by_day.plot()

In [ ]:
cr_by_day_rolling = order_leads.groupby(['Date']) \
    .agg({'Converted': 'mean'}) \
    .rename(columns={'Converted': 'CR'}) \
    .sort_values('Date') \
    .rolling(30).mean()

In [ ]:
ax = cr_by_day_rolling.plot()
ax.set_title('Convertion rate by day\n')
ax.set_ylabel('Convertion rate\n')
ax.set_xlabel('\nDate')
ax.set_yticklabels(['{:.0%}'.format(n) for n in ax.get_yticks()])
sns.despine()

In [ ]:
def get_plot(df, x_label='', y_label='', title=''):
    ax = df.plot()

    ax.set_title(title)
    ax.set_ylabel(y_label)
    ax.set_xlabel(x_label)
    ax.set_yticklabels(['{:.0%}'.format(n) for n in ax.get_yticks()])
    sns.despine()

In [ ]:
get_plot(cr_by_day_rolling)

In [ ]:
cr_by_day_rolling_ = cr_by_day_rolling.reset_index().dropna()

In [ ]:
cr_by_day_rolling_.head()

In [ ]:
fig = px.line(cr_by_day_rolling_, x="Date", y="CR")
fig.show()